In [2]:
import pandas as pd
import os

In [3]:
data_dir = '../data/argumentation/'
LOGIC_dir = '../data/LOGIC'

In [61]:
df_cckg = pd.read_csv(os.path.join(data_dir, 'train_cckg.tsv'), sep='\t')
df_iam = pd.read_csv(os.path.join(data_dir, 'train_iam.tsv'), sep='\t')

In [62]:
examples = pd.read_json(os.path.join(data_dir, 'fallacies_arguments_support.json'))
df_mapping = pd.read_csv(os.path.join(LOGIC_dir, 'mappings.csv'))
df_logic = pd.read_csv(os.path.join(LOGIC_dir, 'train.csv'))

In [63]:
def generate_prompt(topic, fallacy_type, supporting=True):
    data = examples[fallacy_type]
    supporting_args = [data['exampleArg1'], data['exampleArg2']]
    fallacies = [data['example1'], data['example2']]
    
     
    text =  f"""You are given a topic.  
    Your task is to generate a {'supporting' if supporting else 'counter'} {fallacy_type} logical fallacy argument in the context of the topic. 
    It should not be longer than 25 words. 
    
    {fallacy_type} fallacy is defined as: {examples[fallacy_type]['definition']}
    examples of {fallacy_type} fallacy are: 
    {fallacies[0]}
    {fallacies[1]}
    
    Here is an example of a supporting {fallacy_type} fallacy: 
    {supporting_args[0]}
    
    return the following:
    {"{"}
        "topic": {topic},
        "fallacy type": {fallacy_type},
        "{fallacy_type} fallacy support": <> 
    {"}"}
    """
    return text

In [79]:
topic =  df_iam.topic.sample(1).values[0]
fallacy_types = examples.keys()
fallacy_type = fallacy_types[12]
print(fallacy_type)
prompt = generate_prompt(topic=topic, fallacy_type=fallacy_type)
print(topic)
print()
print(prompt)

intentional
Should vaccination be mandatory

You are given a topic.  
    Your task is to generate a supporting intentional logical fallacy argument in the context of the topic. 
    It should not be longer than 25 words. 
    
    intentional fallacy is defined as: some intentional (sometimes subconscious) action/choice to incorrectly support an argument
    examples of intentional fallacy are: 
    Yes, I do think that all drunk drivers should go to prison, but your honor, he is my son! He is a good boy who just made a mistake!
    Bill: I think that some people have psychic powers.  Jill: What is your proof? Bill: No one has been able to prove that people do not have psychic powers.
    
    Here is an example of a supporting intentional fallacy: 
    {'topic': 'Are aliens real', 'fallacy type': 'intentional fallacy', 'intentional fallacy support': "You can't prove that aliens don't exist, so they must be real"}
    
    return 
    {
        "topic": Should vaccination be mandatory